In [1]:
import re
import numpy as np
import pandas as pd
from langdetect import detect

In [2]:
def store_text_splits(text_file_path):
    with open(text_file_path, 'r', encoding='utf-8') as f:
        text=[]
        for line in f:
            splits = line.split()
            if splits !=[]:
                text.append(splits)
    return text

In [3]:
text=store_text_splits('63 Psychology.txt')

In [4]:
for item in text:
    time_marks=[]
    if 'Time' in item:
        for x in item:
            if x.isdigit()==True:
                print(x)

3
70


In [5]:
def req_list(req_string):
    index=[]
    for item in text:
        x=item
        if (set(req_string).issubset(set(x))):
            index.append(text.index(item))
    return index 

In [6]:
time_marks=req_list(['Time'])
time_marks

[27]

In [7]:
code=req_list(['Code', 'No.'])
code_no=text[code[0]][2]
code_no

'63'

In [8]:
g_i=req_list(['Questions','no.','Part'])
g_i

[64, 67, 70, 73, 76]

In [9]:
import re
digit=[]
sub=[]
for i in g_i:
    s=text[i]
    for x in s:
        p=re.compile(r'\d+\-\d+')
        q=p.findall(x)
        if q!=[]:
            digit.append(q)
        if x.isdigit()==True:
            sub.append(x)
digit.append(sub)
digit

[['1-10'], ['11-16'], ['17-20'], ['21-26'], ['27', '28']]

In [10]:
q_range=[]
for i in digit:
    if len(i)==1:
        q_range.append(i[0].split('-'))
    else:
        q_range.append(i)
q_range

[['1', '10'], ['11', '16'], ['17', '20'], ['21', '26'], ['27', '28']]

In [11]:
part=req_list(['PART'])
part_name=[]
part_marks=[]
for i in part:
    part_name.append(text[i][1])
    part_marks.append(text[i][2].replace('(',''))
part_dict={'Part name':part_name,'Part marks':part_marks,'Question_range':digit}
Part=pd.DataFrame(part_dict)
Part

,Part name,Part marks,Question_range
0,A,1,[1-10]
1,B,2,[11-16]
2,C,8,[17-20]
3,D,4,[21-26]
4,E,6,"[27, 28]"


In [12]:
#extract line by line sentence by joining after Part A
sen_list=[]
for item in text:
    if text.index(item)>part[0]:
        sen_list.append(' '.join(item))

# remove unusual question no
remove_item=[]
for x in sen_list:
    p=re.compile(r'\d+\.')
    q=p.findall(x)
    if q!=[]:
        remove_item.append(q)

for i in remove_item:
    sen_list.remove(i[0])

# remove unusual page no and pto
remove_pto=[]
for x in sen_list:
    p=re.compile(r'\d+\ P.T.O.')
    q=p.findall(x)
    if q!=[]:
        remove_pto.append(q)

for i in remove_pto:
    sen_list.remove(i[0])

In [13]:
#separate hindi and english separate two list
hindi=[]
english=[]
h_q=[]
e_q=[]
for s in sen_list:
    if detect(s)!='hi' and detect(s)!='ne' and detect(s)!='mr':
        h_q.append('\n')
        e_q.append(s)
    else:
        e_q.append('\n')
        h_q.append(s)

In [14]:
part_values=[' '.join(text[i]) for i in part]
part_values_hi=[' '.join(text[i-1]) for i in part]

In [15]:
EQ=' '.join(e_q)
EnglishQ=EQ.split('\n')

for i in EnglishQ:
    if i.strip() in part_values:
        EnglishQ.remove(i)

EnglishQ_f=[]
for i in EnglishQ:
    if i.strip():
        EnglishQ_f.append(i)
    

alter_eng=[]
for item in EnglishQ_f:
    if 'OR' in item:
        alter_eng.append(item.split('OR')[1])
        item=item.split('OR')[0]
    else:
        alter_eng.append('NO')

In [16]:
HQ=' '.join(h_q)
HindiQ=HQ.split('\n')

for i in HindiQ:
    if i.strip() in part_values_hi:
        HindiQ.remove(i)

HindiQ_f=[]
for i in HindiQ:
    if i.strip():
        HindiQ_f.append(i)

alter_hi=[]
for item in HindiQ_f:
    if 'अथवा' in item:
        alter_hi.append(item.split('अथवा')[1])
        item=item.split('अथवा')[0]
    else:
        alter_hi.append('NO')



In [17]:
options_eng=[]
for item in EnglishQ_f:
    p=re.compile(r'\(\w+\)')
    q=p.findall(item)
    if q!=[]:
        options_eng.append(item.split('('))

options=[]
for item in EnglishQ_f:
    p=re.compile(r'\(\w+\)')
    q=p.findall(item)
    if q!=[]:
        options.append('yes')
    else:
        options.append('no')


options_hi=[]
for item in HindiQ_f:
    p=re.compile(r'\(\w+\)')
    q=p.findall(item)
    if q!=[]:
        options_hi.append(item.split('('))

In [18]:
marks=[]
for i in range(len(q_range)):
    for j in range(int(q_range[i][0]),int(q_range[i][1])+1):
        marks.append(int(part_marks[i]))

In [19]:
len(marks)

28

In [20]:
EnglishQ_df= pd.DataFrame(EnglishQ_f,columns=['English Question'])
HindiQ_df= pd.DataFrame(HindiQ_f,columns=['Hindi Question'])
questions=pd.concat([EnglishQ_df,HindiQ_df],axis=1)
questions['q_no'] = questions.index+1
questions['Marks']=marks
questions['Alternative_English_Question']=alter_eng
questions['Alternative_Hindi_Question']=alter_hi
questions['Options']=options

In [21]:
option_hi_df=pd.DataFrame(options_hi,columns=['Hindi Question','Option A','Option B','Option C','Option D'])
option_eng_df=pd.DataFrame(options_eng,columns=['English Question','Option A','Option B','Option C','Option D'])

In [22]:
questions

,English Question,Hindi Question,q_no,Marks,Alternative_English_Question,Alternative_Hindi_Question,Options
0,"According to teachers, students who have a co...",अध्यापकों के अनुसार जिन विद्यार्थियों में उच्च...,1,1,Sternberg’s experiential intelligence include...,स्टर्नबर्ग की आनुभविक बुद्धि में सम्मिलित है ...,yes
1,personality is characterised by proneness to ...,अवसाद के प्रति प्रवणता व्यक्तित्व की विशेषता ...,2,1,NO,NO,no
2,Shobhita did not get admission in the college...,शोभिता को अपनी पसंद के कॉलेज में प्रवेश नहीं ...,3,1,NO,NO,no
3,a | A student blames her/his best friend for ...,एक विद्यार्थी अपने खराब प्रदर्शन के लिए अध्यय...,4,1,NO,NO,no
4,Fundamental attribution error is an overall t...,मूल गुणारोपण त्रुटि एक समग्र प्रवृत्ति है जो ...,5,1,An attitude change is said to be congruent wh...,NO,no
5,According to experts who deal with Post Traum...,अभिघातज उत्तर दबाव विकार से निपटने वाले विशेष...,6,1,NO,NO,yes
6,Nishant feels very strongly that everyone sho...,निशांत बहुत चाहता है कि हर व्यक्ति उसे प्यार ...,7,1,NO,NO,yes
7,A student forms her/his impression about a ne...,कोई विद्यार्थी विद्यालय में एक नए अध्यापक के ...,8,1,There will be consistency between attitude an...,जब अभिवृत्ति दुर्बल हो और अभिवृत्ति प्रणाली म...,no
8,"While using psychological tests, an attitude ...",मनोवैज्ञानिक परीक्षणों का उपयोग करते समय | तथ...,9,1,NO,NO,no
9,"After a long discussion, the initial position...",एक लंबे विचार-विमर्श के बाद समूह की प्रारंभिक...,10,1,NO,NO,yes


21s